In [39]:
import spacy
nlp = spacy.load('fr_core_news_md')
import os
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import FrenchStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from collections import Counter
from IPython.display import clear_output
import time
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
import ast

In [40]:
stop_words = set(stopwords.words('french'))
stop_words.add('''['id=""Biographie"">Biographie''')
stop_words.add('''[modifier', '|', 'modifier', 'le', 'code]''')
stop_words.add('''[\'id="Biographie">Biographie[modifier\'''')
stop_words.add('''|''')
stop_words.add(''' ''')
stop_words.add('''\'''')
stop_words.add('''(''')
stop_words.add(''')''')
stop_words.add('''d\\\'''')
stop_words.add('''id="Biographie">Biographie[modifier''')
stop_words.add('''[''')
stop_words.add(''']''')
stop_words.add('nan')

In [41]:
stop_words

{' ',
 "'",
 '(',
 ')',
 '[',
 '[\'id=""Biographie"">Biographie',
 '[\'id="Biographie">Biographie[modifier\'',
 "[modifier', '|', 'modifier', 'le', 'code]",
 ']',
 'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'ces',
 'd',
 "d\\'",
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'id="Biographie">Biographie[modifier',
 'il',
 'ils',
 'j',
 'je',
 'l',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'm',
 'ma',
 'mais',
 'me',
 'mes

In [42]:
stop_sentences = [
 '[\'id=""Biographie"">Biographie',
 '[\'id="Biographie">Biographie[modifier\'',
 "[modifier', '|', 'modifier', 'le', 'code]",
 "[modifier\', \'|\', \'modifier\', \'le\', \'code]",
 "modifier\', \'|\', \'modifier\', \'le\', \'code]",
 '"',
'<span',
' ','[',']','.',',','|',
 "'",
 '(',
 ')',
 '|',]

In [43]:
stop_words

{' ',
 "'",
 '(',
 ')',
 '[',
 '[\'id=""Biographie"">Biographie',
 '[\'id="Biographie">Biographie[modifier\'',
 "[modifier', '|', 'modifier', 'le', 'code]",
 ']',
 'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'ces',
 'd',
 "d\\'",
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'id="Biographie">Biographie[modifier',
 'il',
 'ils',
 'j',
 'je',
 'l',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'm',
 'ma',
 'mais',
 'me',
 'mes

In [45]:
data=pd.read_csv('data/Suicides_wikipedia_scrapping_fr.csv')
#data=pd.read_csv('BiographieParParagraphe.csv',sep=',')

In [46]:
data=pd.DataFrame(data)

In [47]:
list(data)

['Name', 'Link', 'Content']

In [48]:
data['Content'][2]=data['Content'][1]
for j in range(len(stop_sentences)):
    data['Content'][2]=re.sub('\[[\d]\]','',data['Content'][2])
    data['Content'][2]=data['Content'][2].replace(stop_sentences[j],' ')
data['Content'][2]=' '.join(data['Content'][2].split())
data['Content'][2]

'modifier le code Carrière artistique Artiste multidisciplinaire peinture sculpture photographie vidéo il enseigne dans différentes écoles de Tel Aviv depuis 1993 Il est récompensé par le prix du Ministère de l Éducation et de la Culture 2003 et le prix du musée d art de Petah Tikva 2006 Ses œuvres sont présentées dans différents musées dont le musée d Israël à Jérusalem Mort Le 1er février 2018 le site d informations Mako révèle qu il est visé par une enquête concernant des relations sexuelles qu il aurait eues avec des élèves mineures Le lendemain il est retrouvé mort chez lui apparemment suicidé'

In [49]:
data['Content'][2]

'modifier le code Carrière artistique Artiste multidisciplinaire peinture sculpture photographie vidéo il enseigne dans différentes écoles de Tel Aviv depuis 1993 Il est récompensé par le prix du Ministère de l Éducation et de la Culture 2003 et le prix du musée d art de Petah Tikva 2006 Ses œuvres sont présentées dans différents musées dont le musée d Israël à Jérusalem Mort Le 1er février 2018 le site d informations Mako révèle qu il est visé par une enquête concernant des relations sexuelles qu il aurait eues avec des élèves mineures Le lendemain il est retrouvé mort chez lui apparemment suicidé'

In [50]:
for i in range(len(data['Content'])):
    for j in range(len(stop_sentences)):
        data['Content'][i]=re.sub('\[[\d]\]','',data['Content'][i])
        data['Content'][i]=data['Content'][i].replace(stop_sentences[j],' ')
    data['Content'][i]=' '.join(data['Content'][i].split())

In [51]:
data['Content'][1]

'modifier le code Carrière artistique Artiste multidisciplinaire peinture sculpture photographie vidéo il enseigne dans différentes écoles de Tel Aviv depuis 1993 Il est récompensé par le prix du Ministère de l Éducation et de la Culture 2003 et le prix du musée d art de Petah Tikva 2006 Ses œuvres sont présentées dans différents musées dont le musée d Israël à Jérusalem Mort Le 1er février 2018 le site d informations Mako révèle qu il est visé par une enquête concernant des relations sexuelles qu il aurait eues avec des élèves mineures Le lendemain il est retrouvé mort chez lui apparemment suicidé'

In [52]:
data['Content_Lemmatiser']=data['Content']

In [54]:
data.Content_Lemmatiser=[el.split(' ') for el in data.Content_Lemmatiser]


In [57]:
data['Content_Lemmatiser'][i-1]

['modifier',
 'le',
 'code',
 'Markwitz',
 'suivre',
 'un',
 'formation',
 'pratique',
 'et',
 'artisanal',
 'et',
 'être',
 'recommander',
 'dans',
 'le',
 'cinéma',
 'comme',
 'charpentier',
 'talentueux',
 'à',
 'le',
 'fin',
 'de',
 '1931',
 'il',
 'commence',
 'à',
 'travailler',
 'à',
 'titre',
 'mineur',
 'pour',
 'un',
 'production',
 'cinématographique',
 'même',
 'à',
 'Hollywood',
 'au',
 'début',
 'The',
 'Crowd',
 'roar',
 'alia',
 'the',
 'Doctor',
 'en',
 '1934',
 'il',
 'être',
 'autoriser',
 'à',
 'participer',
 'pour',
 'le',
 'premier',
 'fois',
 'à',
 'le',
 'création',
 'de',
 'bâtiment',
 'de',
 'cinéma',
 'en',
 'tant',
 'qu',
 'assister',
 'de',
 'production',
 'et',
 'être',
 'assigner',
 'à',
 'son',
 'collègue',
 'Heinrich',
 'richter',
 'et',
 'Hans',
 'jacoby',
 'le',
 'première',
 'œuvre',
 'de',
 'Markwitz',
 'être',
 'deux',
 'production',
 'd',
 'Erich',
 'Engels',
 'avec',
 'Karl',
 'Valentin',
 'et',
 'Liesl',
 'Karlstadt',
 'dans',
 'le',
 'rôle',
 '

In [56]:
start_time=time.time()
for i in range(len(data['Content_Lemmatiser'])):
    data['Content_Lemmatiser'][i]=data['Content_Lemmatiser'][i]
    print(str(i/len(data['Content_Lemmatiser'])*100)[:4]+'%')
    for j in range(len(data['Content_Lemmatiser'][i])):
        if data['Content_Lemmatiser'][i]!=['']:
            doc = nlp(data['Content_Lemmatiser'][i][j])
            data['Content_Lemmatiser'][i][j]=doc[0].lemma_
        else:
            data['Content_Lemmatiser'][i]=[]
        
    clear_output(wait=True)
print("Temps d execution : %s secondes ---" % (time.time() - start_time))

0.57%


KeyboardInterrupt: 

In [208]:
data['Content_Lemmatiser'][i][3:]

['code',
 'Zusho',
 'être',
 'naître',
 'dans',
 'le',
 'ville',
 'autour',
 'de',
 'château',
 'de',
 'Kagoshima',
 'en',
 '1776',
 'il',
 'être',
 'le',
 'fils',
 'de',
 'Motoaki',
 'Kawasaki',
 'un',
 'samouraï',
 'de',
 'satsuma',
 'à',
 'l',
 'âge',
 'de',
 '12',
 'an',
 'il',
 'être',
 'adopter',
 'par',
 'kiyonobu',
 'Zusho',
 ';',
 'à',
 '22',
 'an',
 'il',
 'être',
 'envoyer',
 'à',
 'edo',
 'pour',
 'devenir',
 'l',
 'assister',
 'de',
 'thé',
 'de',
 'daimyo',
 'retirer',
 'de',
 'satsuma',
 'Shimazu',
 'Shigehide',
 'celui',
 'reconnaître',
 'le',
 'talent',
 'de',
 'Zusho',
 'et',
 'lui',
 'donner',
 'plus',
 'de',
 'responsabilité',
 'il',
 'être',
 'plus',
 'tard',
 'employer',
 'par',
 'le',
 'daimyo',
 'en',
 'service',
 'de',
 'satsuma',
 'Shimazu',
 'Narioki',
 'lui',
 'servir',
 'de',
 'messager',
 'et',
 'de',
 'magistrat',
 'il',
 'être',
 'également',
 'impliquer',
 'dans',
 'de',
 'commerce',
 'illégal',
 'avec',
 'le',
 'Chine',
 'via',
 'le',
 'île',
 'Ryūkyū'

In [209]:
for i in range(len(data['Content_Lemmatiser'])):
    data['Content_Lemmatiser'][i]=data['Content_Lemmatiser'][i][3:]
for i in range(len(data['Content_Lemmatiser_No_Stop_Words'])):
    data['Content_Lemmatiser_No_Stop_Words'][i]=data['Content_Lemmatiser_No_Stop_Words'][i][2:]


In [125]:
for i in range(len(data['Content_Lemmatiser_No_Stop_Words'])):
    data['Content_Lemmatiser_No_Stop_Words'][i]=[w for w in data['Content_Lemmatiser'][i] if w not in stop_words]

In [124]:
stop_words.add('»')
stop_words.add('«')
stop_words.add('etc')
stop_words.add(';')

In [146]:
data['Content_Lemmatiser_No_Stop_Words']=data['Lemmatiser']

In [213]:
data.to_csv('Suicides_wikipedia_scrapping_fr_LEMMATISER.csv',encoding='utf-8',sep=';')